# Introduction

This notebook is a walkthrough of describing 1 of the steps in the ABC Enhancer Gene Prediction model

[broadinstitute/ABC-Enhancer-Gene-Prediction: Cell type specific enhancer-gene predictions using ABC model (Fulco, Nasser et al, Nature Genetics 2019)](https://github.com/broadinstitute/ABC-Enhancer-Gene-Prediction)  

We wrap the macs call peaks step.  The example code in the github repo is this

```
macs2 callpeak \
-t example_chr22/input_data/Chromatin/wgEncodeUwDnaseK562AlnRep1.chr22.bam \
-n wgEncodeUwDnaseK562AlnRep1.chr22.macs2 \
-f BAM \
-g hs \
-p .1 \
--call-summits \
--outdir example_chr22/ABC_output/Peaks/ 

#Sort narrowPeak file
bedtools sort -faidx example_chr22/reference/chr22 -i example_chr22/ABC_output/Peaks/wgEncodeUwDnaseK562AlnRep1.chr22.macs2_peaks.narrowPeak > example_chr22/ABC_output/Peaks/wgEncodeUwDnaseK562AlnRep1.chr22.macs2_peaks.narrowPeak.sorted
```

# Load packages

In [1]:
pip install yamlmagic pyyaml


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sevenbridges import Api, ImportExportState
import time
import json
import importlib
import getpass
import yaml

In [3]:
%load_ext yamlmagic

# Section 1/6 - Tool Label

In [4]:
%%yaml label
label: MACS2 Call Peaks

<IPython.core.display.Javascript object>

# Section 2/6 Tool Inputs

In [5]:
%%yaml inputs

#bam file is main inputs
inputs:
- id: bam
  type: File
  secondaryFiles:
  - pattern: .bai
    required: true
  sbg:fileTypes: BAM

#sort order for bedtools
- id: sort_order
  type: File
  inputBinding:
    position: 0
    shellQuote: false

<IPython.core.display.Javascript object>

# Section 3/6 Base command

In [6]:
%%yaml base_command
baseCommand:
- bash
- call_peaks.sh

<IPython.core.display.Javascript object>

# Section 4/6 Requirements

In [7]:
%%yaml requirements
requirements:
- class: ShellCommandRequirement
- class: DockerRequirement
  dockerPull: images.sb.biodatacatalyst.nhlbi.nih.gov/andrewblair/cardiac-compendium:2023042401
- class: InitialWorkDirRequirement
- class: InitialWorkDirRequirement
  listing:
  - entryname: call_peaks.sh
    writable: false
    entry: |-
      macs2 callpeak -t $(inputs.bam.path) -n $(inputs.bam.nameroot).macs2 -f BAM -g hs -p .1 --call-summits --outdir ./

      #Sort narrowPeak file

      bedtools sort \
      -faidx $(inputs.sort_order.path) \
      -i $(inputs.bam.nameroot).macs2_peaks.narrowPeak \
      > $(inputs.bam.nameroot).macs2_peaks.narrowPeak.sorted
- class: InlineJavascriptRequirement

<IPython.core.display.Javascript object>

# Section 5/6 Outputs

In [8]:
%%yaml outputs
outputs:
- id: model_r_script
  type: File
  outputBinding:
    glob: '*model.r'
- id: sorted_peaks
  type: File
  outputBinding:
    glob: '*narrowPeak.sorted'
- id: peaks_xls
  type: File
  outputBinding:
    glob: '*peaks.xls'
- id: summits
  type: File
  outputBinding:
    glob: '*summits.bed'


<IPython.core.display.Javascript object>

# Section 6/6 Misc settings

In [9]:
%%yaml misc_settings
### below is boiler plate and rarely changes
cwlVersion: v1.2
class: CommandLineTool

$namespaces:
  sbg: https://sevenbridges.com
  
hints:
- class: sbg:SaveLogs
  value: '*.sh'  

<IPython.core.display.Javascript object>

# Push tool to platform

In [10]:
api_token = getpass.getpass()

In [11]:
api = Api(url = "https://api.sb.biodatacatalyst.nhlbi.nih.gov/v2",  token = api_token)

In [12]:
with open('macs2_call_peaks.cwl', 'w') as f:
    data = yaml.dump(label | inputs | base_command | requirements | outputs | misc_settings, f, sort_keys=False, default_flow_style=False)

In [13]:
api.apps.install_app(
    id='dave/abc-development-scratch-project/macs2-call-peaks/19',
    raw=label | inputs | base_command | requirements | outputs | misc_settings)

<App: id=dave/abc-development-scratch-project/macs2-call-peaks rev=19>

# Push this notebook to files tab

In [23]:
upload = api.files.upload(
    path='macs2_call_peaks.cwl.ipynb',
    overwrite=True,
    parent = "644299a6dc22f20baf6998fc",
)


# References

https://docs.sevenbridges.com/reference/add-an-app-using-raw-cwl